This file just tests the bell state measurement function in Trajectree. Due to the integral nature of this function in all measurements, it is important that this works correctly. We will be trying to charecterize a perfect bell state using the function. 

In [1]:
from trajectree.fock_optics.utils import create_vacuum_state
from trajectree.fock_optics.devices import ry, rx, rz, global_phase, generalized_mode_mixer_unitary
from trajectree.fock_optics.measurement import bell_state_measurement
from trajectree.fock_optics.outputs import print_quantum_state, read_quantum_state
from trajectree.fock_optics.noise_models import single_mode_bosonic_noise_channels, depolarizing_operators, two_qubit_depolarizing_channel
from trajectree.fock_optics.light_sources import light_source

from trajectree.trajectory import trajectory_evaluator, quantum_channel

import numpy as np
from scipy import sparse as sp
import qutip as qt
from numpy import sqrt
from quimb.tensor.tensor_arbgeom import tensor_network_apply_op_vec #type: ignore
from quimb.tensor import MatrixProductOperator as mpo #type: ignore

import matplotlib.pyplot as plt
import matplotlib.animation as animation

import copy
from IPython.display import HTML
from IPython.display import Image

In [2]:
def create_perfect_bell_state(N=3, error_tolerance = 1e-10):
    # Defining the control and target inputs here
    a_dag = qt.create(N).full()
    I = qt.identity(N).full()

    a_H = np.kron(a_dag, I)
    a_V = np.kron(I, a_dag)

    # Testing the case with only one MPS for both control and target
    vacuum = create_vacuum_state(num_modes=4, N=N)
    pure_EPPS_operator = 1/sqrt(2) * (np.kron(a_H, a_H) - np.kron(a_V, a_V))
    pure_EPPS_operator_MPO = mpo.from_dense(pure_EPPS_operator, dims = N, sites = (0,1,2,3), L=4, tags="ideal_state")    
    psi = tensor_network_apply_op_vec(pure_EPPS_operator_MPO, vacuum, compress=True, contract = True, cutoff = error_tolerance)
    return psi, psi.site_tags



In [3]:
N = 3
num_modes = 4
efficiency = 1
error_tolerance = 1e-10
num_simulations = 1

bsm_det_effs = [0.1,0.1]
bsm_dark_counts_gain = [1+1e-6,1+1e-6]
# bsm_measurements = {0:(2,3), 1:(0,1)}
bsm_measurements = {0:(0,3), 1:(1,2)}


In [5]:
# psi, site_tags = create_perfect_bell_state(N)

mean_photon_num = 1
vacuum = create_vacuum_state(num_modes=8, N=N)
psi, _ = light_source(vacuum, N, mean_photon_num, 8, error_tolerance, compress=True, contract=True)

read_quantum_state(psi, N, num_states = 2)

Corresponding Basis terms:
0H0V_B 0H0V_A - 0 - [0.52144802+0.j]
1H0V_B 1H0V_A - 10 - [-0.29744465+0.j]
2H0V_B 2H0V_A - 20 - [0.40133111+0.j]
0H1V_B 0H1V_A - 30 - [-0.29744465+0.j]
1H1V_B 1H1V_A - 40 - [0.16966853+0.j]
2H1V_B 2H1V_A - 50 - [-0.2289275+0.j]
0H2V_B 0H2V_A - 60 - [0.40133111+0.j]
1H2V_B 1H2V_A - 70 - [-0.2289275+0.j]
2H2V_B 2H2V_A - 80 - [0.30888345+0.j]


In [ ]:
import time
quantum_channel_list = []

phase_flip_ops = [sp.csr_matrix(global_phase(0, N, return_unitary = True, site1 = 0, site2 = 1, total_sites = 2))]
quantum_channel_list.append(quantum_channel(N = N, num_modes = 4, formalism = "kraus", kraus_ops_tuple = ((0,1), phase_flip_ops), name = "first_kraus"))

BSM_quantum_channels = bell_state_measurement(None, N, psi.site_tags, num_modes, bsm_det_effs, bsm_dark_counts_gain, error_tolerance, beamsplitters = [[0,2],[1,3]], measurements = bsm_measurements, pnr = False, use_trajectory = True, return_MPOs = True, compress=True, contract=True)
quantum_channel_list.extend(BSM_quantum_channels)

evaluator = trajectory_evaluator(quantum_channel_list)

times = []
for i in range(1):
    start = time.time()
    print(i)
    # psi_iter = evaluator.perform_simulation(psi, error_tolerance, trajectree_indices = [0,1,0,0,0], normalize = True)
    psi_iter = evaluator.perform_simulation(psi, error_tolerance, normalize = True)
    duration = time.time() - start
    times.append(duration)

    # evaluator.graph.show()
    
read_quantum_state(psi_iter, N, num_states = 2)
# plt.plot(times)
# psi_iter.normalize()
# coincidence_HH_ops_list

0
Corresponding Basis terms:
1H0V_B 0H2V_A - 15 - [-0.04026813+0.j]
1H2V_B 0H2V_A - 69 - [0.+1.e-10j]


In [15]:
import io
evaluator.graph.show()
evaluator.graph.to_dict()
# display(Image(dot_data.pipe(format='png')))

()
└── (0,)
    └── (0, 0)
        └── (0, 0, 0)
            └── (0, 0, 0, 0)



{'()': {'children': [{'(0,)': {'children': [{'(0, 0)': {'children': [{'(0, 0, 0)': {'children': ['(0, 0, 0, 0)']}}]}}]}}]}}

In [16]:
for qc in quantum_channel_list:
    print(qc.name)


first_kraus
beam splitter
beam splitter
detector inefficiency
detector inefficiency
dark counts
dark counts
Det POVM
Det POVM
Det POVM
Det POVM
